# Stage 03a: Generate Plots for Detailed Evolution

This notebook demonstrates the process of generating plots for individual system properties using the outputs from previous pipeline stages. It is adapted from the `Stage_03a_generate_plots.py` script in the detailed evolution folder.

In [1]:
import os, sys
import numpy as np
import numpy.ma as ma
import pandas as pd
import logging
import time as t
import datetime as dt
import h5py as h5
import matplotlib.pyplot as plt
import matplotlib
from astropy.io import fits
from astropy.table import Table
from astropy import constants as const
from astropy import units as u
from luxetenebrae import calculations as calc
from luxetenebrae import utils as utils

now = dt.datetime.now()
mode = 'WD_Enabled_Detailed'

compasRootDir = os.environ['COMPAS_ROOT_DIR']
sys.path.append(compasRootDir + '/postProcessing/PythonScripts')
print("COMPAS_ROOT_DIR:", compasRootDir)
print(sys.path)

pathToData = os.path.join(compasRootDir, "luxetenebrae/runs", mode)
pathToFiles = os.path.join(compasRootDir, "luxetenebrae/files", mode)
pathToLogs = os.path.join(compasRootDir, "luxetenebrae/logs", mode)
pathToPlots = os.path.join(compasRootDir, "luxetenebrae/plots", mode)
os.makedirs(pathToFiles, exist_ok=True)
os.makedirs(pathToLogs, exist_ok=True)
os.makedirs(pathToPlots, exist_ok=True)

  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018
  Name   = Solar mass
  Value  = 1.988409870698051e+30
  Uncertainty  = 4.468805426856864e+25
  Unit  = kg
  Reference = IAU 2015 Resolution B 3 + CODATA 2018
solMass
1.988409870698051e+30 kg
1.988409870698051e+30
0.43249345526948063 AU
176223120.3149359 s
[0.05714286 0.0625     0.06666667]
[0.05862051 0.31034555 0.78246195 ... 0.67951969 0.63705141 0.9443498 ]
1175
percentage of searchability: 0.001175
1.5707940987948965
1.5706217938696971
0.00017453292431338717
COMPAS_ROOT_DIR: /data/a.saricaoglu/repo/COMPAS
['/grad/a.saricaoglu/.conda/envs/compas/lib/python312.zip', '/grad/a.saricaoglu/.conda/envs/compas/lib/python3.12', '/grad/a.saricaoglu/.conda/envs/compas/lib/python3.12/lib-dynload', '', '/grad/a.saricaoglu/.conda/envs/compas/lib/python3.12/site-packages', '/data/a.saricaoglu/repo/COMPAS/luxetenebrae/src', '/data/a.saricaoglu/repo/COMPAS/postProcessin

In [2]:
script_name = 'Stage_03a_generate_plots.ipynb'
log_filename = f"{pathToLogs}/{script_name}_{now.strftime('%m.%d')}_{now.strftime('%H%M')}.log"
os.makedirs(os.path.dirname(log_filename), exist_ok=True)
logging.basicConfig(filename=log_filename, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Displays Time
s = dt.datetime.now()
starttime = t.ctime(t.time())
start = t.process_time()
start_time = s.strftime("%d%m%y") + "_" + s.strftime('%H%M')
print("Start time :", start_time)

Start time : 300625_1412


In [3]:
out = [f for f in os.listdir(pathToFiles) if "stage_02_outputs" in f]
print('Output files found:', out)
detailed_output = []
SP_output = []
MT_output = []
CE_output = []
for f in out:
    print("Reading file: ", pathToFiles + "/" + f)
    data = fits.open(pathToFiles + "/" + f)
    print("Number of tables in the file:", len(data))
    detailed_output.append(data[1])
    SP_output.append(data[2])
    if len(data) > 3:
        MT_output.append(data[3])
    if len(data) > 4:
        CE_output.append(data[4])

Output files found: ['stage_02_outputs_1751049448.fits', 'stage_02_outputs_1751049414.fits', 'stage_02_outputs_1751049442.fits', 'stage_02_outputs_1751049463.fits', 'stage_02_outputs_1751049452.fits', 'stage_02_outputs_1751049456.fits', 'stage_02_outputs_1751049475.fits', 'stage_02_outputs_1751049419.fits', 'stage_02_outputs_1751049480.fits', 'stage_02_outputs_1751049484.fits', 'stage_02_outputs_1751049662.fits', 'stage_02_outputs_1751049507.fits', 'stage_02_outputs_1751049501.fits', 'stage_02_outputs_1751049505.fits', 'stage_02_outputs_1751049654.fits', 'stage_02_outputs_1751049420.fits', 'stage_02_outputs_1751049412.fits', 'stage_02_outputs_1751049667.fits', 'stage_02_outputs_1751049425.fits', 'stage_02_outputs_1751049673.fits', 'stage_02_outputs_1751049678.fits', 'stage_02_outputs_1751049681.fits', 'stage_02_outputs_1751049731.fits', 'stage_02_outputs_1751049745.fits', 'stage_02_outputs_1751049910.fits', 'stage_02_outputs_1751049922.fits', 'stage_02_outputs_1751049928.fits', 'stage_

## Process and Plot Each System

For each system in the output files, extract relevant data and generate plots for stellar type evolution, semi-major axis, mass transfer history, and eccentricity.

In [4]:
from matplotlib import ticker

i = 0
# Here, we collect merger flags for different types of mergers.
merger_flags_manual = []
merger_flags_mthist = []
merger_flags = []

# Here, we collect final systems for BH-MS and BH-BH pairs.
bhms_final_systems = []
bhbh_final_systems = []

# Here, we collect average parameter data for population characteristics.
semimajoraxis_average = []
lifetime_average = []
eccentricity_average = []

for Data, Data_SP in zip(detailed_output, SP_output):
    Data = Data.data
    Data_SP = Data_SP.data
    seed = Data['Seed'][0]
    print('seed', seed)
    stellar_type_1 = Data['Stellar_Type_1']
    stellar_type_2 = Data['Stellar_Type_2']
    radius_1 = (Data['Radius(1)']*const.R_sun).to(u.au).value
    radius_2 = (Data['Radius(2)']*const.R_sun).to(u.au).value
    time = Data['Time']
    semimajoraxis = Data['SemiMajorAxis']
    roche_lobe_1 = (Data['RocheLobe(1)']*const.R_sun).to(u.au).value
    roche_lobe_2 = (Data['RocheLobe(2)']*const.R_sun).to(u.au).value
    mass_1 = Data['Mass(1)']
    mass_2 = Data['Mass(2)']
    mass_zams_1 = Data['Mass@ZAMS(1)']
    mass_zams_2 = Data['Mass@ZAMS(2)']
    mt_history = Data['MT_History']
    eccentricity = Data['Eccentricity']
    periapsiss = calc.periapsis(semimajoraxis, eccentricity)
    merger = Data_SP['Merger']

    mask = np.array([utils.is_ms_bh_pair(st1, st2) for st1, st2 in zip(stellar_type_1, stellar_type_2)])
    merger_flag_manual = False
    for r1, r2, sa, e in zip(radius_1, radius_2, semimajoraxis, eccentricity):
        if utils.check_merger_manual(r1, r2, sa, e):
            print('check merger manual True')
            merger_flag_manual = True
    if merger_flag_manual:       
        merger_flags_manual.append(seed)

    merger_flag_mthist = False
    if utils.check_merger_MT_hist(mt_history):
        print('check merger mt hist True')
        merger_flag_mthist = True
        print('r1 + r2 =', radius_1[-1] + radius_2[-1], 'semimajoraxis =', semimajoraxis[-1])
        merger_flags_mthist.append(seed)

    if utils.is_ms_bh_pair(stellar_type_1[-1], stellar_type_2[-1]):
        bhms_final_systems.append(seed)
    if utils.is_bh_bh_pair(stellar_type_1[-1], stellar_type_2[-1]):
        bhbh_final_systems.append(seed)
    merger_flags.append(merger)

    time_filtered = Data['Time'][:][mask]
    stype1_filtered = stellar_type_1[mask]
    stype2_filtered = stellar_type_2[mask]
    radius1_filtered = radius_1[mask]
    radius2_filtered = radius_2[mask]
    semimajoraxis_filtered = semimajoraxis[mask]
    eccentricity_filtered = eccentricity[mask]
    
    bhms_lifetime = time_filtered[-1] - time_filtered[0]
    bhms_life = '{:.2f}'.format(bhms_lifetime) 
    semaj_average = np.mean(semimajoraxis_filtered)
    semaj_ave= '{:.2f}'.format(semaj_average) 

    bhms_lifetime = time_filtered[-1] - time_filtered[0]
    lifetime_average.append(bhms_lifetime) 

    semaj_average = np.mean(semimajoraxis_filtered)
    semimajoraxis_average.append(semaj_average)

    ecc_average = np.mean(eccentricity_filtered)
    eccentricity_average.append(ecc_average)

    fig, ax = plt.subplots(4,1, sharex=True, figsize=(4, 10))
    ax[0].scatter(time, stellar_type_1, s=2, label='Primary Star', color='blue')
    ax[0].scatter(time, stellar_type_2, s=2, label='Secondary Star', color='red')
    ax[0].set_yticks(np.arange(0,18,1), [r'MS (M<0.7 $M_\odot$)', r'MS (M>0.7 $M_\odot$)', 'HG', 'FGB', 'CHeB', 'EAGB', 'TPAGB', 'HeMS', 'HeHG', 'HeGB', 'HeWD', 'COWD', 'ONeWD','NS', 'BH', 'MR', 'CHE', 'None'])
    ax[0].fill_between(time_filtered, 0, np.max(stype1_filtered) , color='orange', alpha=0.1)
    ax[0].text(0.05, 0.95, fr'Lifetime of BH-MS: {bhms_life} Myr', transform=ax[0].transAxes,  verticalalignment='top')
    ax[0].legend()
    ax[0].grid(False) 
    ax[0].set_ylabel('Stellar Type')

    ax[1].plot(time, semimajoraxis, label='Semi-major Axis', color='yellow')
    ax[1].plot(time, radius_1, label=r'$R_p$', color='blue')
    ax[1].plot(time, radius_2, label=r'$R_s$', color='red')
    ax[1].plot(time, roche_lobe_1, label=r'$Roche R_p$', color='blue', linestyle='dashed')
    ax[1].plot(time, roche_lobe_2, label=r'$Roche R_s$', color='red', linestyle='dashed')   
    ax[1].plot(time, periapsiss, label='Periapsis', color='yellow', linestyle='dashed')
    ax[1].legend()
    ax[1].grid(False) 
    ax[1].fill_between(time_filtered, 0, max(np.max(semimajoraxis_filtered), np.max(radius_1), np.max(radius_2)), color='orange', alpha=0.1)
    ax[1].text(0.05, 0.15, rf'$\langle$SA$\rangle_\tau$: {semaj_ave} AU', transform=ax[1].transAxes,  verticalalignment='top')
    ax[1].set_ylabel('Semi-major Axis [AU]')

    ax[2].scatter(time, mt_history, s=2)
    ax[2].fill_between(time_filtered, 0, 7, color='orange', alpha=0.1)
    ax[2].set_yticks(np.arange(0,7,1), ['No MT', 'MT 1->2', 'MT 2->1', 'MTCE 1->2', 'MTCE 2->1', 'MTCE DoubleCore', 'Merger'])
    ax[2].text(0.05, 0.95, f'Merger (from flag): {merger}', transform=ax[2].transAxes,  verticalalignment='top')
    ax[2].text(0.05, 0.85, f'Merger (from mthist): {merger_flag_mthist}', transform=ax[2].transAxes,  verticalalignment='top')
    ax[2].text(0.05, 0.75, f'Merger (manual): {merger_flag_manual}', transform=ax[2].transAxes,  verticalalignment='top')
    ax[2].grid(False) 
    ax[2].set_ylabel('MT History')

    ax[3].plot(time, eccentricity, label='Eccentricity', color='yellow')
    ax[3].grid(False) 
    ax[3].set_ylabel('Eccentricity')        

    plt.xlabel('Time [Myr]')
    plt.legend()
    plt.grid(False) 
    plt.savefig(f'{pathToPlots}/stellar_type_evolution_{i}.png',bbox_inches='tight')
    plt.close()
    i = i + 1

seed 1751049448
seed 1751049414
seed 1751049414
seed 1751049442
seed 1751049442
seed 1751049463
seed 1751049463
seed 1751049452
seed 1751049452
seed 1751049456
seed 1751049456
seed 1751049475
seed 1751049475
seed 1751049419
seed 1751049419
seed 1751049480
seed 1751049480
seed 1751049484
seed 1751049484
seed 1751049662
seed 1751049662
seed 1751049507
check merger mt hist True
r1 + r2 = 0.0023361255977294723 semimajoraxis = 0.004788330730048401
seed 1751049507
check merger mt hist True
r1 + r2 = 0.0023361255977294723 semimajoraxis = 0.004788330730048401
seed 1751049501
check merger mt hist True
r1 + r2 = 0.04065817527863699 semimajoraxis = 0.07743775854937149
seed 1751049501
check merger mt hist True
r1 + r2 = 0.04065817527863699 semimajoraxis = 0.07743775854937149
seed 1751049505
check merger manual True
check merger manual True
check merger manual True
check merger manual True
check merger mt hist True
r1 + r2 = 4.545052484712636 semimajoraxis = 0.018678896536380506
seed 1751049505
che

## Plot Population Stats

Here we plot the parameter distribution of the whole population. No need to run the first cell if you run the cell that produces individual plots.

In [ ]:
from matplotlib import ticker

i = 0
# Here, we collect average parameter data for population characteristics.
semimajoraxis_average = []
lifetime_average = []
eccentricity_average = []

for Data, Data_SP in zip(detailed_output, SP_output):
    Data = Data.data
    Data_SP = Data_SP.data
    seed = Data['Seed'][0]
    print('seed', seed)
    stellar_type_1 = Data['Stellar_Type_1']
    stellar_type_2 = Data['Stellar_Type_2']
    radius_1 = (Data['Radius(1)']*const.R_sun).to(u.au).value
    radius_2 = (Data['Radius(2)']*const.R_sun).to(u.au).value
    time = Data['Time']
    semimajoraxis = Data['SemiMajorAxis']
    roche_lobe_1 = (Data['RocheLobe(1)']*const.R_sun).to(u.au).value
    roche_lobe_2 = (Data['RocheLobe(2)']*const.R_sun).to(u.au).value
    mass_1 = Data['Mass(1)']
    mass_2 = Data['Mass(2)']
    mass_zams_1 = Data['Mass@ZAMS(1)']
    mass_zams_2 = Data['Mass@ZAMS(2)']
    mt_history = Data['MT_History']
    eccentricity = Data['Eccentricity']
    periapsiss = calc.periapsis(semimajoraxis, eccentricity)
    merger = Data_SP['Merger']

    mask = np.array([utils.is_ms_bh_pair(st1, st2) for st1, st2 in zip(stellar_type_1, stellar_type_2)])
    merger_flag_manual = False
    for r1, r2, sa, e in zip(radius_1, radius_2, semimajoraxis, eccentricity):
        if utils.check_merger_manual(r1, r2, sa, e):
            print('check merger manual True')
            merger_flag_manual = True
    if merger_flag_manual:       
        merger_flags_manual.append(seed)

    merger_flag_mthist = False
    if utils.check_merger_MT_hist(mt_history):
        print('check merger mt hist True')
        merger_flag_mthist = True
        print('r1 + r2 =', radius_1[-1] + radius_2[-1], 'semimajoraxis =', semimajoraxis[-1])
        merger_flags_mthist.append(seed)

    if utils.is_ms_bh_pair(stellar_type_1[-1], stellar_type_2[-1]):
        bhms_final_systems.append(seed)
    if utils.is_bh_bh_pair(stellar_type_1[-1], stellar_type_2[-1]):
        bhbh_final_systems.append(seed)
    merger_flags.append(merger)

    time_filtered = Data['Time'][:][mask]
    stype1_filtered = stellar_type_1[mask]
    stype2_filtered = stellar_type_2[mask]
    radius1_filtered = radius_1[mask]
    radius2_filtered = radius_2[mask]
    semimajoraxis_filtered = semimajoraxis[mask]
    eccentricity_filtered = eccentricity[mask]
    
    bhms_lifetime = time_filtered[-1] - time_filtered[0]
    bhms_life = '{:.2f}'.format(bhms_lifetime) 
    semaj_average = np.mean(semimajoraxis_filtered)
    semaj_ave= '{:.2f}'.format(semaj_average) 

    bhms_lifetime = time_filtered[-1] - time_filtered[0]
    lifetime_average.append(bhms_lifetime) 

    semaj_average = np.mean(semimajoraxis_filtered)
    semimajoraxis_average.append(semaj_average)

    ecc_average = np.mean(eccentricity_filtered)
    eccentricity_average.append(ecc_average)

    i += 1

In [5]:
fig, ax = plt.subplots(3,1, figsize=(3.5, 7))
ax[0].hist(semimajoraxis_average, bins=50, color='blue', alpha=0.7, label='Semimajor Axis')
ax[0].set_xlabel('Semimajor Axis (AU)')
ax[0].set_ylabel('Number of Systems')
ax[0].set_yscale('log')
ax[1].hist(lifetime_average, bins=50, color='green', alpha=0.7, label='Lifetime')
ax[1].set_xlabel('Lifetime (Myr)')
ax[1].set_ylabel('Number of Systems')
ax[1].set_yscale('log')
ax[2].hist(eccentricity_average, bins=50, color='red', alpha=0.7, label='Eccentricity')
ax[2].set_xlabel('Eccentricity')
ax[2].set_ylabel('Number of Systems')
ax[2].set_yscale('log')
#plt.suptitle('BH-MS Systems Average Histogram', fontsize=16)
plt.tight_layout()
plt.savefig(f'{pathToPlots}/bhms_properties_histogram.png', bbox_inches='tight')
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3.5))
# counts, bin_edges1 = np.histogram(primary_mass, bins=50)
# counts, bin_edges2 = np.histogram(secondary_mass, bins=50)
# counts1, bin_edges = np.histogram(survivor_primary_mass, bins=50)
# counts2, bin_edges = np.histogram(survivor_secondary_mass, bins=50)
# survivor_primary_mass_density = density(counts1, bins=bin_edges1)
# survivor_secondary_mass_density = density(counts2, bins=bin_edges2)
ax.hist(primary_mass, bins=50, color='blue', alpha=0.7, label='Primary Mass', histtype='step')
ax.hist(survivor_primary_mass, bins=50, color='blue', alpha=0.7, label='BHMS Primary Mass', histtype='step', linestyle='--')
ax.hist(secondary_mass, bins=50, color='orange', alpha=0.7, label='Secondary Mass', histtype='step')
ax.hist(survivor_secondary_mass, bins=50, color='orange', alpha=0.7, label='BHMS Secondary Mass', histtype='step', linestyle='--')
ax.set_xlabel('Mass (M☉)')
ax.set_ylabel('Number of Systems')
ax.set_yscale('log')
ax.legend()
#ax.set_title('Initial Mass Distribution of BH-MS Systems')
plt.tight_layout()
plt.savefig(directoryp + 'bhms_mass_distribution_initial.png')
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3.5))
ax.hist(primary_mass, bins=50, color='blue', alpha=0.7, label='Primary Mass', density=True)
ax.hist(secondary_mass, bins=50, color='orange', alpha=0.7, label='Secondary Mass', density=True)
kroupa_fit = kroupa_imf_normalized(5, 150)
ax.plot(np.linspace(5, 150, 1000), kroupa_fit, color='black', linestyle='--', label='Kroupa IMF')
ax.set_xlabel('Mass (M☉)')
ax.set_ylabel('Probability Density')
ax.set_yscale('log')
ax.legend()
plt.tight_layout()
plt.savefig(directoryp + 'bhms_mass_distribution_density.png')
plt.close()

## Summary

This notebook has processed the detailed evolution outputs and generated plots for each system, including stellar type evolution, semi-major axis, mass transfer history, and eccentricity. The generated plots are saved in the specified output directory.

In [ ]:
print(f'Systems with merger (R1+R2>SA): {(merger_flags_manual)}')
print(f'Systems with merger (MT hist): {(merger_flags_mthist)}')
print(f'Systems with merger (flag): {(merger_flags)}')
print(f'Systems with BH-MS final state:{(bhms_final_systems)}')



print(f'Number of systems with merger (R1+R2>SA): {len(merger_flags_manual)}')
print(f'Number of systems with merger (MT hist): {len(merger_flags_mthist)}')
print(f'Number of systems with merger (flag): {np.sum(merger_flags)}')
print(f'Number of systems with BH-MS final state:{len(bhms_final_systems)}')
print(f'Number of systems with BH-BH final state:{len(bhbh_final_systems)}')

print(f'Number of systems with primary BH: {primary_bh}')
print(f'Number of systems with secondary BH: {secondary_bh}')
print(f'Total number of systems with BHs: {i}')
print(f'Total number of systems with BH-MS pairs: {len(lifetime_average)}')

print(f'Skipped systems due to absence of BH-MS pairs: {j}')
print(f'Skipped systems due to negative semimajor axis: {k}')

print(f'Minimum primary mass: {np.min(survivor_primary_mass)} M☉')
print(f'Maximum primary mass: {np.max(survivor_primary_mass)} M☉')